In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
import json
import time
import pandas as pd

# Input and output file paths
input_file_path = "/content/tuba_members_all.json"  # Replace with actual file path
output_file_path = "tuba_author_ids.json"

# Load KNAW members JSON
with open(input_file_path, 'r', encoding='utf-8') as file:
    members = json.load(file)

# OpenAlex API endpoint
openalex_base_url = "https://api.openalex.org/authors"

# Function to query OpenAlex
def get_openalex_author_id(name):
    try:
        response = requests.get(openalex_base_url, params={"search": name})
        response.raise_for_status()  # Raise an exception for HTTP errors
        results = response.json()

        if "results" in results and results["results"]:
            # Take the first match (or apply more filters if needed)
            first_result = results["results"][0]
            return {
                "author_id": first_result.get("id"),
                "name": first_result.get("display_name"),
                "works_count": first_result.get("works_count", 0),
                "cited_by_count": first_result.get("cited_by_count", 0),
            }
        else:
            return None  # No match found
    except Exception as e:
        print(f"Error fetching author ID for {name}: {e}")
        return None

# Process each member to fetch OpenAlex Author ID
results = []
for member in members:
    print(f"Fetching OpenAlex Author ID for {member['Name']}...")
    author_data = get_openalex_author_id(member["Name"])
    if author_data:
        results.append({
            "Name": member["Name"],
            "Institution": member["Institution"],
            "Department": member["Department"],
            "Academic Title": member["Academic Title"],
            "openalex_id": author_data["author_id"],
            "works_count": author_data["works_count"],
            "cited_by_count": author_data["cited_by_count"],
        })
    else:
        results.append({
            "Name": member["Name"],
            "Institution": member["Institution"],
            "Department": member["Department"],
            "Academic Title": member["Academic Title"],
            "openalex_id": None,
            "works_count": None,
            "cited_by_count": None,
        })
    # Respect API rate limits
    time.sleep(1)  # Adjust delay if needed

# Save the results to a JSON file
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"Author IDs saved to '{output_file_path}'.")

# Optionally, save to CSV for easier inspection
pd.DataFrame(results).to_csv("tuba_author_ids.csv", index=False)
print("Author IDs also saved to 'tuba_author_ids.csv'.")

Fetching OpenAlex Author ID for Muzaffer Şeker...
Fetching OpenAlex Author ID for Atilla Abdulkadiroğlu...
Fetching OpenAlex Author ID for Mustafa Acar...
Fetching OpenAlex Author ID for Ahmet Cevat Acar...
Fetching OpenAlex Author ID for Alparslan Açıkgenç...
Fetching OpenAlex Author ID for Nazmi Volkan Adsay...
Fetching OpenAlex Author ID for Ali Ekber Akgün...
Fetching OpenAlex Author ID for M. İrşadi Aksun...
Fetching OpenAlex Author ID for Yasin Aktay...
Fetching OpenAlex Author ID for Şener Aktürk...
Fetching OpenAlex Author ID for Ali Akyıldız...
Fetching OpenAlex Author ID for Mehmet Hakkı Alma...
Fetching OpenAlex Author ID for Meliha Altunışık...
Fetching OpenAlex Author ID for M. Fatih Andı...
Fetching OpenAlex Author ID for Mustafa Reşat Apak...
Fetching OpenAlex Author ID for Erol Arcaklıoğlu...
Fetching OpenAlex Author ID for Metin Arık...
Fetching OpenAlex Author ID for Erdal Arıkan...
Fetching OpenAlex Author ID for Hüseyin Arslan...
Fetching OpenAlex Author ID for Ercü

In [ ]:
import requests
import json
import time

# Input and output file paths
input_file = "/content/tuba_author_ids.json"  # Replace with your actual file path
output_file = "tuba_members_with_indices.json"  # File to save the updated data

# OpenAlex base API URL
openalex_base_url = "https://api.openalex.org"

# Function to fetch H-index and I10-index for an author
def fetch_author_metrics(author_id):
    url = f"{openalex_base_url}/authors/{author_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        h_index = data.get("summary_stats", {}).get("h_index", None)
        i10_index = data.get("summary_stats", {}).get("i10_index", None)
        return h_index, i10_index
    else:
        print(f"Error fetching data for author {author_id}: {response.status_code}")
        return None, None

# Load the existing JSON file
with open(input_file, "r") as infile:
    members = json.load(infile)

# Process each member and fetch H-index and I10-index
for member in members:
    openalex_ids = member.get("openalex_id")
    if isinstance(openalex_ids, list):
        # Initialize variables to aggregate metrics
        h_indices = []
        i10_indices = []
        for openalex_id in openalex_ids:
            author_id = openalex_id.split("/")[-1]  # Extract author ID
            h_index, i10_index = fetch_author_metrics(author_id)
            if h_index is not None:
                h_indices.append(h_index)
            if i10_index is not None:
                i10_indices.append(i10_index)
            time.sleep(1)  # Respect API rate limits

        # Aggregate indices (e.g., take maximum values)
        member["h_index"] = max(h_indices) if h_indices else None
        member["i10_index"] = max(i10_indices) if i10_indices else None

    elif isinstance(openalex_ids, str):
        author_id = openalex_ids.split("/")[-1]  # Extract author ID
        h_index, i10_index = fetch_author_metrics(author_id)
        member["h_index"] = h_index
        member["i10_index"] = i10_index
        time.sleep(1)  # Respect API rate limits
    else:
        print(f"No OpenAlex ID for {member['Name']}. Skipping...")
        member["h_index"] = None
        member["i10_index"] = None

# Save the updated JSON with indices
with open(output_file, "w") as outfile:
    json.dump(members, outfile, indent=4)

print(f"H-index and I10-index added and saved to {output_file}")

No OpenAlex ID for Atilla Abdulkadiroğlu. Skipping...
No OpenAlex ID for M.talha Çiçek. Skipping...
No OpenAlex ID for Prof. Dr. Robert Dankoff. Skipping...
No OpenAlex ID for Ayşe Selçuk Esenbel. Skipping...
No OpenAlex ID for K. Arzum  Erdem Gürsan. Skipping...
No OpenAlex ID for Ahmet Saim Kılavuz. Skipping...
No OpenAlex ID for Jeffry David Sachs. Skipping...
H-index and I10-index added and saved to tuba_members_with_indices.json


In [ ]:
import json

# Input and output file paths
input_file = "/content/tuba_members_with_indices.json"  # Replace with your distorted JSON file path
output_file = "tuba_members_with_indices_restored.json"  # Restored output file

# Load the distorted JSON file
with open(input_file, "r", encoding="utf-8") as infile:
    data = json.load(infile)

# Decode Unicode escape sequences
def decode_unicode(data):
    if isinstance(data, dict):
        return {key: decode_unicode(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [decode_unicode(item) for item in data]
    elif isinstance(data, str):
        try:
            # Attempt to decode the string
            return data.encode('latin1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            # Return the original string if decoding fails
            return data
    else:
        return data

restored_data = decode_unicode(data)

# Save the restored JSON file
with open(output_file, "w", encoding="utf-8") as outfile:
    json.dump(restored_data, outfile, ensure_ascii=False, indent=4)

print(f"Restored file saved to {output_file}")

Restored file saved to tuba_members_with_indices_restored.json


In [ ]:
import requests
import json
import time
import pandas as pd

# Input and output file paths
input_file = "/content/tuba_members_with_indices_restored.json"  # Replace with your actual file path
output_json_file = "tuba_works_filtered.json"  # JSON file output
output_csv_file = "tuba_works_filtered.csv"  # CSV file output
progress_log_file = "progress_log4.json"  # Log file to track progress

# OpenAlex base API URL
openalex_base_url = "https://api.openalex.org"

# Function to fetch works for an author
def fetch_author_works(author_id):
    works = []
    page = 1
    while True:
        url = f"{openalex_base_url}/works?filter=author.id:{author_id}&per-page=200&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for work in data.get("results", []):  # Handle missing 'results'
                filtered_work = {
                    "id": work.get("id", ""),
                    "title": work.get("title", ""),
                    "doi": work.get("doi", ""),
                    "publication_year": work.get("publication_year", ""),
                    "cited_by_count": work.get("cited_by_count", 0),
                    "authorships": [
                        {
                            "author": {
                                "id": auth.get("author", {}).get("id", ""),
                                "display_name": auth.get("author", {}).get("display_name", "")
                            },
                            "institutions": [
                                {
                                    "id": inst.get("id", ""),
                                    "display_name": inst.get("display_name", ""),
                                    "country_code": inst.get("country_code", ""),
                                    "type": inst.get("type", "")  # Safely handle missing type
                                }
                                for inst in auth.get("institutions", []) if inst
                            ]
                        }
                        for auth in work.get("authorships", []) if auth
                    ],
                    "primary_topic": {
                        "id": work.get("primary_topic", {}).get("id", ""),
                        "display_name": work.get("primary_topic", {}).get("display_name", ""),
                        "score": work.get("primary_topic", {}).get("score", 0),
                        "field": {
                            "id": work.get("primary_topic", {}).get("field", {}).get("id", ""),
                            "display_name": work.get("primary_topic", {}).get("field", {}).get("display_name", "")
                        } if work.get("primary_topic", {}).get("field") else {},
                        "subfield": {
                            "id": work.get("primary_topic", {}).get("subfield", {}).get("id", ""),
                            "display_name": work.get("primary_topic", {}).get("subfield", {}).get("display_name", "")
                        } if work.get("primary_topic", {}).get("subfield") else {},
                        "domain": {
                            "id": work.get("primary_topic", {}).get("domain", {}).get("id", ""),
                            "display_name": work.get("primary_topic", {}).get("domain", {}).get("display_name", "")
                        } if work.get("primary_topic", {}).get("domain") else {}
                    } if work.get("primary_topic") else {},  # Handle missing primary_topic
                    "concepts": [
                        {
                            "id": concept.get("id", ""),
                            "display_name": concept.get("display_name", ""),
                            "level": concept.get("level", 0),
                            "score": concept.get("score", 0)
                        }
                        for concept in work.get("concepts", []) if concept
                    ],
                    "open_access": work.get("open_access", {}),
                    "sustainable_development_goals": [
                        {
                            "id": sdg.get("id", ""),
                            "score": sdg.get("score", 0),
                            "display_name": sdg.get("display_name", "")
                        }
                        for sdg in work.get("sustainable_development_goals", []) if sdg
                    ],
                    "referenced_works": work.get("referenced_works", [])
                }
                works.append(filtered_work)
            if "next" not in data.get("meta", {}):  # Handle missing 'meta'
                break
            page += 1
            time.sleep(1)  # Respect API rate limits
        else:
            print(f"Error fetching works for author {author_id}: {response.status_code}")
            break
    return works

# Load the existing JSON file
with open(input_file, "r") as infile:
    members = json.load(infile)

# Load or initialize the progress log
try:
    with open(progress_log_file, "r") as log_file:
        progress_log = json.load(log_file)
except FileNotFoundError:
    progress_log = {}

# Fetch works for each author and organize them under authors
authors_with_works = []
for member in members:
    name = member["Name"]
    openalex_ids = member.get("openalex_id")
    all_works = []

    if name in progress_log and progress_log[name]:
        print(f"Skipping {name}, already completed.")
        continue

    print(f"Fetching works for {name}...")

    try:
        if isinstance(openalex_ids, list):
            for openalex_id in openalex_ids:
                author_id = openalex_id.split("/")[-1]  # Extract author ID
                all_works.extend(fetch_author_works(author_id))
        elif isinstance(openalex_ids, str):
            author_id = openalex_ids.split("/")[-1]
            all_works.extend(fetch_author_works(author_id))

        authors_with_works.append({
            "name": name,
            "openalex_id": openalex_ids,
            "works": all_works
        })

        # Mark the author as successfully processed
        progress_log[name] = True

    except Exception as e:
        print(f"Error processing {name}: {e}")
        progress_log[name] = False  # Mark as failed

    # Save progress after each author
    with open(progress_log_file, "w") as log_file:
        json.dump(progress_log, log_file, indent=4)

# Save the filtered data to a JSON file
with open(output_json_file, "w") as outfile:
    json.dump(authors_with_works, outfile, indent=4)
print(f"Filtered works data saved to JSON: {output_json_file}")

# Convert to CSV for easier inspection
csv_data = []
for author in authors_with_works:
    for work in author["works"]:
        csv_data.append({
            "author_name": author["name"],
            "author_openalex_id": author["openalex_id"],
            "work_id": work["id"],
            "work_title": work["title"],
            "doi": work["doi"],
            "publication_year": work["publication_year"],
            "cited_by_count": work["cited_by_count"],
            "field": work["primary_topic"].get("field", {}).get("display_name", ""),
            "subfield": work["primary_topic"].get("subfield", {}).get("display_name", ""),
            "domain": work["primary_topic"].get("domain", {}).get("display_name", ""),
            "concepts": ", ".join([concept["display_name"] for concept in work["concepts"]]),
            "open_access_status": work.get("open_access", {}).get("oa_status", ""),
            "sustainable_development_goals": ", ".join([sdg["display_name"] for sdg in work.get("sustainable_development_goals", [])]),
            "referenced_works_count": len(work.get("referenced_works", []))
        })

pd.DataFrame(csv_data).to_csv(output_csv_file, index=False)
print(f"Filtered works data saved to CSV: {output_csv_file}")

Fetching works for Muzaffer Şeker...
Fetching works for Atilla Abdulkadiroğlu...
Fetching works for Mustafa Acar...
Fetching works for Ahmet Cevat Acar...
Fetching works for Alparslan Açıkgenç...
Fetching works for Nazmi Volkan Adsay...
Fetching works for Ali Ekber Akgün...
Fetching works for M. İrşadi Aksun...
Fetching works for Yasin Aktay...
Fetching works for Şener Aktürk...
Fetching works for Ali Akyıldız...
Fetching works for Mehmet Hakkı Alma...
Fetching works for Meliha Altunışık...
Fetching works for M. Fatih Andı...
Fetching works for Mustafa Reşat Apak...
Fetching works for Erol Arcaklıoğlu...
Fetching works for Metin Arık...
Fetching works for Erdal Arıkan...
Fetching works for Hüseyin Arslan...
Fetching works for Ercümend Arvas...
Fetching works for Messoud Ashina...
Fetching works for Abdullah Atalar...
Fetching works for Ali Tayfun Atay...
Fetching works for Abdurrahman Atçıl...
Fetching works for Cenk Ayata...
Fetching works for Orhan Aydın...
Fetching works for Mehmet 

In [ ]:
import json

# Input and output file paths
input_file = "/content/tuba_works_filtered.json"  # Replace with your distorted JSON file path
output_file = "tuba_works_filtered_restored.json"  # Restored output file

# Load the distorted JSON file
with open(input_file, "r", encoding="utf-8") as infile:
    data = json.load(infile)

# Decode Unicode escape sequences
def decode_unicode(data):
    if isinstance(data, dict):
        return {key: decode_unicode(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [decode_unicode(item) for item in data]
    elif isinstance(data, str):
        try:
            # Attempt to decode the string
            return data.encode('latin1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            # Return the original string if decoding fails
            return data
    else:
        return data

restored_data = decode_unicode(data)

# Save the restored JSON file
with open(output_file, "w", encoding="utf-8") as outfile:
    json.dump(restored_data, outfile, ensure_ascii=False, indent=4)

print(f"Restored file saved to {output_file}")

Restored file saved to tuba_works_filtered_restored.json


In [ ]:
import pandas as pd

# Input and output file paths
input_file = "/content/tuba_works_filtered.csv"  # Replace with your distorted CSV file path
output_file = "tuba_works_filtered_restored.csv"  # Restored output file

# Load the distorted CSV file
df = pd.read_csv(input_file)

# Decode Unicode escape sequences in strings
def decode_unicode_string(value):
    if isinstance(value, str):  # Ensure the value is a string before decoding
        try:
            return value.encode('latin1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            return value  # Return the original value if decoding fails
    return value  # Return the value as-is if it's not a string

# Apply the Unicode decoding function to all string cells in the DataFrame
for col in df.columns:
    if df[col].dtype == 'object':  # Apply only to object (string) columns
        df[col] = df[col].apply(decode_unicode_string)

# Save the restored CSV file
df.to_csv(output_file, index=False)

print(f"Restored file saved to {output_file}")

Restored file saved to tuba_works_filtered_restored.csv
